In [1]:
import pandas as pd
import numpy as np
from keras.models import Model
from keras.layers import Input, LSTM, Dense
from keras.callbacks import EarlyStopping

Using TensorFlow backend.


In [2]:
df = pd.read_csv('../data/eng-katakana.csv', header=None, names=['eng', 'katakana'])

In [3]:
df.head()

,eng,katakana
0,Unschooling,アンスクーリング
1,Lovosice,ロヴォシツェ
2,Milch,ミルヒ
3,Juva,ユヴァ
4,Brembilla,ブレンビッラ


In [4]:
df.shape

(107261, 2)

In [5]:
df.values

array([['Unschooling', 'アンスクーリング'],
       ['Lovosice', 'ロヴォシツェ'],
       ['Milch', 'ミルヒ'],
       ...,
       ['Buckwheat Zydeco', 'バックウィート・ザディコ'],
       ['Castaway Cay', 'キャスタウェイ・ケイ'],
       ['Astatine', 'アスタチン']], dtype=object)

In [6]:
input_texts = list(df['eng'].values)

In [7]:
input_characters = set()
for name in input_texts:
    for char in name:
        if char not in input_characters:
            input_characters.add(char)
input_characters = sorted(list(input_characters))

In [8]:
print(input_texts[:5])
print(len(input_characters))
print(input_characters)

['Unschooling', 'Lovosice', 'Milch', 'Juva', 'Brembilla']
78
[' ', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', 'ê', 'ò', 'ó', 'õ', 'ù', 'ú', 'ü', 'ý', 'þ', 'ļ', 'ľ', 'ŵ', 'ź', 'ż', 'ž']


In [9]:
target_texts = list(df['katakana'].values)

In [10]:
target_texts[0]

'アンスクーリング'

In [11]:
example = target_texts[:3]
print(example)
example = ['\t' + text + '\n' for text in example]
print(example)

['アンスクーリング', 'ロヴォシツェ', 'ミルヒ']
['\tアンスクーリング\n', '\tロヴォシツェ\n', '\tミルヒ\n']


In [12]:
# Add \t and \n to target_texts
target_texts = ['\t' + text + '\n' for text in target_texts]    

In [13]:
target_characters = set()
for name in target_texts:
    for char in name:
        if char not in target_characters:
            target_characters.add(char)
target_characters = sorted(list(target_characters))

In [14]:
print(target_texts[:5])
print(len(target_characters))
print(target_characters)

['\tアンスクーリング\n', '\tロヴォシツェ\n', '\tミルヒ\n', '\tユヴァ\n', '\tブレンビッラ\n']
89
['\t', '\n', ' ', 'ァ', 'ア', 'ィ', 'イ', 'ゥ', 'ウ', 'ェ', 'エ', 'ォ', 'オ', 'カ', 'ガ', 'キ', 'ギ', 'ク', 'グ', 'ケ', 'ゲ', 'コ', 'ゴ', 'サ', 'ザ', 'シ', 'ジ', 'ス', 'ズ', 'セ', 'ゼ', 'ソ', 'ゾ', 'タ', 'ダ', 'チ', 'ヂ', 'ッ', 'ツ', 'ヅ', 'テ', 'デ', 'ト', 'ド', 'ナ', 'ニ', 'ヌ', 'ネ', 'ノ', 'ハ', 'バ', 'パ', 'ヒ', 'ビ', 'ピ', 'フ', 'ブ', 'プ', 'ヘ', 'ベ', 'ペ', 'ホ', 'ボ', 'ポ', 'マ', 'ミ', 'ム', 'メ', 'モ', 'ャ', 'ヤ', 'ュ', 'ユ', 'ョ', 'ヨ', 'ラ', 'リ', 'ル', 'レ', 'ロ', 'ヮ', 'ワ', 'ヰ', 'ヱ', 'ヲ', 'ン', 'ヴ', '・', 'ー']


In [15]:
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(name) for name in input_texts])
max_decoder_seq_length = max([len(name) for name in target_texts])

In [16]:
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

Number of samples: 107261
Number of unique input tokens: 78
Number of unique output tokens: 89
Max sequence length for inputs: 85
Max sequence length for outputs: 47


In [17]:
# find which name has length 85
for name in input_texts:
    if len(name) > 80:
        print(name)
        print(len(name))
        print(input_texts.index(name))

Taumatawhakatangihangakoauauotamateaturipukakapikimaungahoronukupokaiwhenuakitanatahu
85
98703


In [18]:
target_texts[98703]

'\tタウマタファカタンギハンガコアウアウオタマテアポカイフェヌアキタナタフ\n'

In [19]:
# Token to index dictionary
input_token_index = dict((char, i) for i, char in enumerate(input_characters))
target_token_index = dict((char, i) for i, char in enumerate(target_characters))

In [20]:
print(input_token_index)
print(target_token_index)

{' ': 0, '0': 1, '1': 2, '2': 3, '3': 4, '4': 5, '5': 6, '6': 7, '7': 8, '8': 9, '9': 10, 'A': 11, 'B': 12, 'C': 13, 'D': 14, 'E': 15, 'F': 16, 'G': 17, 'H': 18, 'I': 19, 'J': 20, 'K': 21, 'L': 22, 'M': 23, 'N': 24, 'O': 25, 'P': 26, 'Q': 27, 'R': 28, 'S': 29, 'T': 30, 'U': 31, 'V': 32, 'W': 33, 'X': 34, 'Y': 35, 'Z': 36, 'a': 37, 'b': 38, 'c': 39, 'd': 40, 'e': 41, 'f': 42, 'g': 43, 'h': 44, 'i': 45, 'j': 46, 'k': 47, 'l': 48, 'm': 49, 'n': 50, 'o': 51, 'p': 52, 'q': 53, 'r': 54, 's': 55, 't': 56, 'u': 57, 'v': 58, 'w': 59, 'x': 60, 'y': 61, 'z': 62, 'ê': 63, 'ò': 64, 'ó': 65, 'õ': 66, 'ù': 67, 'ú': 68, 'ü': 69, 'ý': 70, 'þ': 71, 'ļ': 72, 'ľ': 73, 'ŵ': 74, 'ź': 75, 'ż': 76, 'ž': 77}
{'\t': 0, '\n': 1, ' ': 2, 'ァ': 3, 'ア': 4, 'ィ': 5, 'イ': 6, 'ゥ': 7, 'ウ': 8, 'ェ': 9, 'エ': 10, 'ォ': 11, 'オ': 12, 'カ': 13, 'ガ': 14, 'キ': 15, 'ギ': 16, 'ク': 17, 'グ': 18, 'ケ': 19, 'ゲ': 20, 'コ': 21, 'ゴ': 22, 'サ': 23, 'ザ': 24, 'シ': 25, 'ジ': 26, 'ス': 27, 'ズ': 28, 'セ': 29, 'ゼ': 30, 'ソ': 31, 'ゾ': 32, 'タ': 33, 'ダ': 34,

In [21]:
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

In [22]:
print(encoder_input_data[0])

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [23]:
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i, t, input_token_index[char]] = 1.
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.

In [24]:
batch_size = 64  # Batch size for training.
epochs = 100  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.

In [25]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

In [26]:
# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

Becasue training this model on a CPU takes too loog, so here we just train it 2 epoches to see the result. And we will train it on GPU and using early stopping. 

In [27]:
# Run training
earlystopper = EarlyStopping(monitor='val_loss', patience=5, verbose=1)

model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=2,
          validation_split=0.2,
          callbacks=[earlystopper])

Train on 85808 samples, validate on 21453 samples
Epoch 1/2
85808/85808 [==============================] - 605s 7ms/step - loss: 0.6213 - val_loss: 0.5954
Epoch 2/2
85808/85808 [==============================] - 531s 6ms/step - loss: 0.5690 - val_loss: 0.5723


In [82]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, 78)     0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, None, 87)     0                                            
__________________________________________________________________________________________________
lstm_1 (LSTM)                   [(None, 256), (None, 343040      input_2[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  352256      input_3[0][0]                    
                                                                 lstm_1[0][1]                     
          

In [28]:
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence

In [30]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Unschooling
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Lovosice
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Milch
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Juva
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Brembilla
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Sa Pa
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Brumano
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Brusaporto
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Deventer
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Enschede
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Tandil
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Buckypaper
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Bastiglia
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Personalization
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Mandalgovi
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Bomporto
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Campogalliano
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Haa

### GPU training with early stopping

We train the whole model on a GPU and using the CuDNNLSTM in Keras, it is very fast. And we save the weights as `s2s.h5`. 


```
Train on 85808 samples, validate on 21453 samples
Epoch 1/100
85808/85808 [==============================] - 84s 975us/step - loss: 0.6239 - val_loss: 0.5950
Epoch 2/100
85808/85808 [==============================] - 78s 907us/step - loss: 0.5706 - val_loss: 0.5731
Epoch 3/100
85808/85808 [==============================] - 78s 906us/step - loss: 0.5522 - val_loss: 0.5604
Epoch 4/100
85808/85808 [==============================] - 77s 893us/step - loss: 0.5395 - val_loss: 0.5502
Epoch 5/100
85808/85808 [==============================] - 76s 889us/step - loss: 0.5298 - val_loss: 0.5439
Epoch 6/100
85808/85808 [==============================] - 76s 889us/step - loss: 0.5219 - val_loss: 0.5404
Epoch 7/100
85808/85808 [==============================] - 76s 889us/step - loss: 0.5153 - val_loss: 0.5357
Epoch 8/100
85808/85808 [==============================] - 76s 887us/step - loss: 0.5095 - val_loss: 0.5347
Epoch 9/100
85808/85808 [==============================] - 76s 888us/step - loss: 0.5044 - val_loss: 0.5327
Epoch 10/100
85808/85808 [==============================] - 76s 888us/step - loss: 0.4998 - val_loss: 0.5316
Epoch 11/100
85808/85808 [==============================] - 76s 888us/step - loss: 0.4956 - val_loss: 0.5309
Epoch 12/100
85808/85808 [==============================] - 76s 888us/step - loss: 0.4917 - val_loss: 0.5309
Epoch 13/100
85808/85808 [==============================] - 76s 888us/step - loss: 0.4882 - val_loss: 0.5310
Epoch 14/100
85808/85808 [==============================] - 76s 888us/step - loss: 0.4849 - val_loss: 0.5308
Epoch 15/100
85808/85808 [==============================] - 76s 888us/step - loss: 0.4818 - val_loss: 0.5316
Epoch 16/100
85808/85808 [==============================] - 76s 888us/step - loss: 0.4788 - val_loss: 0.5322
Epoch 17/100
85808/85808 [==============================] - 76s 887us/step - loss: 0.4761 - val_loss: 0.5337
Epoch 00017: early stopping
```

In [32]:
# Define an input sequence and process it.
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='categorical_crossentropy')

# Load wegihts from file
model.load_weights('s2s.h5')

In [36]:
for seq_index in range(20):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

-
Input sentence: Unschooling
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Lovosice
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Milch
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Juva
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Brembilla
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Sa Pa
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Brumano
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Brusaporto
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Deventer
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Enschede
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Tandil
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Buckypaper
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Bastiglia
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Personalization
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Mandalgovi
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Bomporto
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Campogalliano
Decoded sentence: アンドレイ・ストリー

-
Input sentence: Haa